In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rand, lit
import random

In [0]:
# Criar sessão Spark
spark = SparkSession.builder.appName("SorteioNegativados").getOrCreate()

# Criar dataframe de exemplo com pessoas negativadas
dados_pessoas = [
    (1, "João Silva", "123.456.789-00", 1500.00),
    (2, "Maria Souza", "987.654.321-00", 2500.00),
    (3, "Carlos Pereira", "321.654.987-00", 3500.00)
]

colunas = ["id", "nome", "cpf", "valor_divida"]
df_pessoas = spark.createDataFrame(dados_pessoas, colunas)

In [0]:
# Salvar em Delta Lake (caso necessário)
df_pessoas.write.format("delta").mode("overwrite").saveAsTable("pessoas_negativadas")


In [0]:
# Criar baralho de descontos (10% a 100%)
descontos = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [0]:
# Sortear pessoa aleatória
pessoa_sorteada = df_pessoas.orderBy(rand()).limit(1).collect()[0]

In [0]:
# Sortear desconto aleatório
desconto_sorteado = random.choice(descontos)

# Calcular novo valor da dívida
novo_valor = pessoa_sorteada["valor_divida"] * (1 - desconto_sorteado / 100)

# Criar DataFrame com resultado
df_sorteio = spark.createDataFrame([(pessoa_sorteada["id"], pessoa_sorteada["nome"], desconto_sorteado, novo_valor)], 
                                   ["id", "nome", "desconto_aplicado", "novo_valor_divida"])

# Salvar no Delta Lake
df_sorteio.write.format("delta").mode("append").saveAsTable("historico_sorteios")

# Exibir resultado
df_sorteio.show()

+---+----------+-----------------+-----------------+
| id|      nome|desconto_aplicado|novo_valor_divida|
+---+----------+-----------------+-----------------+
|  1|João Silva|               20|           1200.0|
+---+----------+-----------------+-----------------+



In [0]:
%sql
SELECT * FROM historico_sorteios ORDER BY id DESC;


id,nome,desconto_aplicado,novo_valor_divida
1,João Silva,20,1200.0
